In [95]:
#von Mises strain calculation
from sympy import *
from sympy.abc import theta, phi
from sympy.functions.special.spherical_harmonics import Ynm
#from sympy.abc import i, j
#from sympy import Function
#from sympy import I
from sympy.tensor import IndexedBase
from sympy.tensor.tensor import TensorIndexType, Tensor, tensorhead, tensor_indices, tensor_heads
from sympy.tensor.array import Array, tensorproduct 

#symbols
l = 2
m = 2
beta = sqrt(l*(l+1))
theta, phi, r = symbols('theta phi r')
i, j = symbols('i j', integer=True)
Y_lm = Ynm(l, m, theta, phi)
#radial functions
W = Function('W')(r)
V = Function('V')(r)
U = Function('U')(r)
T_1 = Function('T_1')(r)
T_2 = Function('T_2')(r)
T_3 = Function('T_3')(r)
mu = Function('mu')(r)
#displacement vectorsf 
xi_r = W*Y_lm
xi_theta = V*(r/beta)*diff(Y_lm, theta) - I*(U/sin(theta))*(r/beta)*diff(Y_lm, phi)
xi_phi = V*(r/beta)*diff(Y_lm, phi) + I*sin(theta)*U*(r/beta)*diff(Y_lm, theta)
#metric tensor
g = Function('g')(i, j, r, theta)
g = diag(1, r**2, (r**2)*(sin(theta)**2))
ginv = Function('ginv')(i, j, r, theta)
ginv = diag(1, 1/(r**2), 1/((r**2)*(sin(theta)**2)))
divxi = (-3*(W/r) - beta*(V/(2*r)) + ((3*T_1)/(4*mu)))*Y_lm


In [64]:
Lorentz = TensorIndexType('Lorentz', dummy_fmt='L')
i, j = tensor_indices('i j', Lorentz)
B = tensorhead('B', [Lorentz]*2, [[1]*2])

/var/folders/2x/9z58jdkj64v7y0p4b_g_d2pr0000gn/T/ipykernel_17272/1431144275.py:1: SymPyDeprecationWarning: 

The dummy_fmt keyword to TensorIndexType is deprecated. Use
dummy_name=L instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-tensorindextype-dummy-fmt
for details.

This has been deprecated since SymPy version 1.5. It
will be removed in a future version of SymPy.

  Lorentz = TensorIndexType('Lorentz', dummy_fmt='L')
/var/folders/2x/9z58jdkj64v7y0p4b_g_d2pr0000gn/T/ipykernel_17272/1431144275.py:3: SymPyDeprecationWarning: 

The tensorhead() function is deprecated. Use tensor_heads() instead.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-tensorhead
for details.

This has been deprecated since SymPy version 1.5. It
will be removed in a future version of SymPy.

  B = tensorhead('B', [Lorentz]*2, [[1]*2])


In [66]:
print(B)

B(Lorentz,Lorentz)


In [96]:
#stress tensor
#A = Tensor('A', indices=[i,j])
A_components = symbols('A_ij')
A_00 = T_1*Y_lm
A_01 = T_2*diff(Y_lm, theta) - I*(T_3/sin(theta))*diff(Y_lm, phi)
A_02 = T_2*diff(Y_lm, phi) + I*sin(theta)*T_3*diff(Y_lm, theta)
A_10 = T_2*diff(Y_lm, theta) - I*(T_3/sin(theta))*diff(Y_lm, phi)
A_20 = T_2*diff(Y_lm, phi) + I*sin(theta)*T_3*diff(Y_lm, theta)
A_11 = 2*diff(xi_theta, theta) - (2/3)*g[1,1]*divxi
A_22 = 2*diff(xi_phi, phi) - (2/3)*g[2,2]*divxi
A_12 = diff(xi_theta, phi) + diff(xi_phi, theta)
A_21 = diff(xi_theta, phi) + diff(xi_phi, theta)
A_compoenets = Matrix([[A_00, A_01, A_02],[A_10, A_11, A_12],[A_20, A_21, A_22]])


In [97]:
A = Matrix([[A_00, A_01, A_02], [A_10, A_11, A_12], [A_20, A_21, A_22]])

display(A)

Matrix([
[                                                             T_1(r)*Ynm(2, 2, theta, phi),                                                                                                                                                                                                                                                                                2*T_2(r)*cot(theta)*Ynm(2, 2, theta, phi) + 2*T_3(r)*Ynm(2, 2, theta, phi)/sin(theta),                                                                                                                                                                                                                                         2*I*T_2(r)*Ynm(2, 2, theta, phi) + 2*I*T_3(r)*sin(theta)*cot(theta)*Ynm(2, 2, theta, phi)],
[    2*T_2(r)*cot(theta)*Ynm(2, 2, theta, phi) + 2*T_3(r)*Ynm(2, 2, theta, phi)/sin(theta), -0.666666666666667*r**2*(3*T_1(r)/(4*mu(r)) - sqrt(6)*V(r)/(2*r) - 3*W(r)/r)*Ynm(2, 2, theta, phi) + 2*sqrt(6)*r*(-cot(theta)

In [90]:
result = tensorcontraction(A*ginv*A, (0,1))
simp_result = simplify(expand(result))

In [94]:
display(simp_result/256)

-0.0009765625*r**2*T_1(r)**2*cos(theta)**2*Ynm(2, 2, theta, phi)**2/mu(r)**2 + 0.001953125*r**2*T_1(r)**2*Ynm(2, 2, theta, phi)**2/mu(r)**2 + 0.0078125*sqrt(6)*r*T_1(r)*U(r)*cos(theta)*Ynm(2, 2, theta, phi)**2/(mu(r)*sin(theta)**2) + 0.00520833333333333*sqrt(6)*r*T_1(r)*U(r)*cos(theta)**5*Ynm(2, 2, theta, phi)**2/(mu(r)*sin(theta)**4) - 0.00520833333333333*sqrt(6)*r*T_1(r)*U(r)*cos(theta)*Ynm(2, 2, theta, phi)**2/(mu(r)*sin(theta)**4) + 0.00130208333333333*sqrt(6)*r*T_1(r)*V(r)*cos(theta)**2*Ynm(2, 2, theta, phi)**2/mu(r) + 0.0078125*sqrt(6)*r*T_1(r)*V(r)*Ynm(2, 2, theta, phi)**2/mu(r) - 0.00260416666666667*sqrt(6)*r*T_1(r)*V(r)*Ynm(2, 2, theta, phi)**2/(mu(r)*sin(theta)**2) + 0.0078125*r*T_1(r)*W(r)*cos(theta)**2*Ynm(2, 2, theta, phi)**2/mu(r) - 0.015625*r*T_1(r)*W(r)*Ynm(2, 2, theta, phi)**2/mu(r) + T_1(r)**2*Ynm(2, 2, theta, phi)**2/256 - 0.03125*T_2(r)**2*Ynm(2, 2, theta, phi)**2 + 0.015625*T_2(r)**2*Ynm(2, 2, theta, phi)**2/sin(theta)**2 + 6.36065274524829e-18*T_2(r)**2*Ynm(2, 2, 

In [25]:
#Calculation
result = A[i,j]*ginv[i,j]*ginv[i,j]*A[i,j]
#result = (A_compoenets*ginv*ginv*A_compoenets)
simp_result = simplify(result)
display(simp_result)

NameError: name 'A' is not defined